# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [3]:
using Pkg
Pkg.add(PackageSpec(name="MLDataUtils"))

   Updating registry at `C:\Users\Marc Juang\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
  Installed MappedArrays ── v0.2.2
  Installed LearnBase ───── v0.3.0
  Installed MLLabelUtils ── v0.5.2
  Installed MLDataUtils ─── v0.5.1
  Installed MLDataPattern ─ v0.5.3
   Updating `C:\Users\Marc Juang\.julia\environments\v1.4\Project.toml`
  [cc2ba9b6] + MLDataUtils v0.5.1
   Updating `C:\Users\Marc Juang\.julia\environments\v1.4\Manifest.toml`
  [7f8f8fb0] + LearnBase v0.3.0
  [9920b226] + MLDataPattern v0.5.3
  [cc2ba9b6] + MLDataUtils v0.5.1
  [66a33bbf] + MLLabelUtils v0.5.2
  [dbb5928d] + MappedArrays v0.2.2


In [4]:
using GLM, RDatasets, MLDataUtils

┌ Info: Precompiling MLDataUtils [cc2ba9b6-d476-5e6d-8eaf-a92d5412d41d]
└ @ Base loading.jl:1260


## 讀取資料

In [2]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [5]:
boston[!, :LogMedV] = log.(boston[!, :MedV])

506-element Array{Float64,1}:
 3.1780538303479458
 3.0726933146901194
 3.5467396869528134
 3.5085558999826545
 3.5890591188317256
 3.3568971227655755
 3.131136910560194
 3.299533727885655
 2.803360380906535
 2.9391619220655967
 2.70805020110221
 2.9391619220655967
 3.077312260546414
 ⋮
 3.1986731175506815
 3.139832617527748
 2.9806186357439426
 2.9069010598473755
 3.054001181677967
 2.8622008809294686
 2.8213788864092133
 3.109060958860994
 3.0252910757955354
 3.173878458937465
 3.091042453358316
 2.4765384001174837

In [6]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [7]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.07244,60.0,1.69,0,0.411,5.884,18.5,10.7103,4,411
2,9.96654,0.0,18.1,0,0.74,6.485,100.0,1.9784,24,666
3,9.72418,0.0,18.1,0,0.74,6.406,97.2,2.0651,24,666
4,0.57529,0.0,6.2,0,0.507,8.337,73.3,3.8384,8,307
5,0.13914,0.0,4.05,0,0.51,5.572,88.5,2.5961,5,296
6,0.1396,0.0,8.56,0,0.52,6.167,90.0,2.421,5,384
7,0.19539,0.0,10.81,0,0.413,6.245,6.2,5.2873,4,305
8,0.30347,0.0,7.38,0,0.493,6.312,28.9,5.4159,5,287
9,0.13642,0.0,10.59,0,0.489,5.891,22.3,3.9454,4,277


In [8]:
model = lm(@formula(LogMedV ~ Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

LogMedV ~ 1 + Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                 Estimate   Std. Error    t value  Pr(>|t|)     Lower 95%     Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)   3.86581      0.223944      17.2624     <1e-49   3.42553       4.30608
Crim         -0.0092264    0.00132517    -6.96242    <1e-10  -0.0118317    -0.00662111
Chas          0.0774603    0.0378375      2.04719    0.0413   0.00307173    0.151849
NOx          -0.778455     0.156354      -4.97879    <1e-6   -1.08585      -0.471062
Rm            0.123659     0.018551       6.66586    <1e-10   0.0871872     0.16013
Dis          -0.0431843    0.00710958    -6.0

In [9]:
exp.(predict(model, test))

101-element Array{Float64,1}:
 18.245840290013213
 15.681670090107948
 15.232501616685623
 40.16834408108821
 22.025352138850735
 20.480981221430156
 24.510084426164628
 24.228658729858374
 22.454647479546303
 32.72555738213874
 18.857534948989198
 32.52844640178049
 13.212629585721185
  ⋮
 13.63020055345036
 18.117374022199847
 40.911238665206
 19.405508472629776
 27.26267717550554
 15.425818780289198
 27.910461142449453
 11.04661915783427
 23.311802283012966
 22.280294696436563
 15.92974867308786
 14.657684445454933

In [10]:
adjr²(model)

0.789075653087854